# Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import re

from bs4 import BeautifulSoup
from datetime import datetime

# 1. Project Plan

In this section, we created the plan to answer the questions asked by the partners. The questions were answered using *SAPE* method. *SAPE* is the method created by a Brazilian Data Scientist (Meigarom Lopes) in order to better organize the strategies to solve the business problems. The most appropriate translation of *SAPE* into English is *OPI*: output, process and input. 

#### 1. What is the best jeans sale price?

Output

1. How to answer the question.

- Median of the products from the competitors website.

2. Format

- Table or chart.

3. Mean of delivery.

- Streamlit App.

Process

1. Steps to calculate the answer.

- Price median per category, type and color.

2. What we will use to create the table and the chart.

- Simulation using Google Sheets.

3. How the final product will be.

- A dashboard with a Streamlit App. It will be published to Heroku (cloud environment).

Input

1. H&M: https://www2.hm.com/en_us/men/products/jeans.html

2. Macys: https://www.macys.com/shop/mens-clothing/mens-jeans

#### 2. How many different types of jeans and colors should we choose?

This question must be answered by the previous plan. 

#### 3. What raw materials should we choose to make the jeans?  

We may answer this question by the same procedure chosen in the previous plan and selecting the composition on the websites. 

# 2. Data Collection 

In this section, we collected the attributes - defined in the previous section - from the American competitors website: **H&M** and **Macys**.

## 2.1. H&M

### 2.1.1 Id, product name, product type/category, price and datetime. 

The attributes collected in this section are part of the showcase for men jeans from H&M website page - but datetime. 

In [3]:
def get_webpage(url01, headers):
    page = requests.get(url01, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')

    return soup
    
def collect_data(soup):    
    products = soup.find('ul', 'products-listing small')
    
    product_id_category = products.find_all('article', 'hm-product-item')
    product_name = products.find_all('a', 'link')
    product_price = products.find_all('span', 'price regular')
    
    return product_id_category, product_name, product_price

def get_id(product_id_category):
    product_id = [p.get('data-articlecode') for p in product_id_category]
    
    return product_id

def get_category(product_id_category):
    product_category = [p.get('data-category') for p in product_id_category]
    
    return product_category

def get_name(product_name):
    product_name = [p.get_text() for p in product_name]
    
    return product_name

def get_price(product_price):
    product_price = [p.get_text() for p in product_price]

    return product_price

def create_dataframe(product_id, product_name, product_category, product_price):
    data = pd.DataFrame([product_id, product_name, product_category, product_price]).T
    data.columns = ['id', 'product_name', 'product_type', 'price']
    
    return data

def set_datetime():
    data['datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return data

if __name__ == '__main__':
    url01 = "https://www2.hm.com/en_us/men/products/jeans.html?sort=stock&image-size=small&image=model&offset=0&page-size=72"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    
    soup = get_webpage(url01, headers)
    
    product_id_category, product_name, product_price = collect_data(soup)
    
    product_id = get_id(product_id_category)
    
    product_category = get_category(product_id_category)
    
    product_name = get_name(product_name)
    
    product_price = get_price(product_price)
    
    data = create_dataframe(product_id, product_name, product_category, product_price)
    
    data = set_datetime()

In [4]:
data

,id,product_name,product_type,price,datetime
0,0875105018,Relaxed Jeans,men_jeans_relaxed,$ 29.99,2022-01-27 11:27:44
1,1008549004,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 11:27:44
2,1008549001,Regular Jeans,men_jeans_regular,$ 19.99,2022-01-27 11:27:44
3,0979945001,Loose Jeans,men_jeans_loose,$ 29.99,2022-01-27 11:27:44
4,0875105016,Relaxed Jeans,men_jeans_relaxed,$ 29.99,2022-01-27 11:27:44
...,...,...,...,...,...
66,0974202002,Regular Denim Joggers,men_jeans_loose,$ 29.99,2022-01-27 11:27:44
67,0985197004,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-27 11:27:44
68,0993887002,Hybrid Regular Denim Joggers,men_jeans_regular,$ 44.99,2022-01-27 11:27:44
69,0927964002,Regular Tapered Crop Jeans,men_jeans_regular,$ 19.99,2022-01-27 11:27:44
